#### 1. 필요한 라이브러리 및 패키지 임포트

In [ ]:
pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 52.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import re
import ast
import fitz
from kiwipiepy import Kiwi
import glob

warnings.filterwarnings('ignore')

#### 2. PDF to 텍스트 변환

In [ ]:
# 경로의 모든 pdf 파일을 읽고 텍스트로 변환하는 함수
def extract_text_from_pdfs(directory_path):
    all_text = [] # 추출 텍스트 저장
    file_names = [] # 처리된 pdf 파일 이름 저장

    # 지정된 디렉토리의 모든 파일 및 하위 디렉토리 탐색
    for root, dirs, files in os.walk(directory_path): 
        for filename in files:
            if filename.endswith(".pdf"): # 파일 확장자가 pdf만 처리
                filepath = os.path.join(root, filename)
                try:
                    file_names.append(filename) # 파일 이름 추가
                
                    doc = fitz.open(filepath) # pdf 열기
                    doc_text = "" # 현재 pdf 문서의 텍스트를 저장하는 변수

                    # 모든 페이지에서 텍스트 추출
                    for page in doc:
                        text = page.get_text() # 현재 페이지에서 텍스트 추출
                        doc_text += text # 추출된 텍스트 추가

                    # 추출된 텍스트를 리스트에 추가
                    all_text.append(doc_text)
                    doc.close() # PDF 닫기(fitz  객체 닫기)

                except Exception as e: # 예외처리
                    print(f"Error processing {filename}: {e}")
    
    return file_names, all_text

In [ ]:
# PDF 파일이 저장된 디렉토리 경로 지정
directory_path = "YOUR PATH"

# PDF 파일에서 텍스트 추출
file_names, all_texts = extract_text_from_pdfs(directory_path)

# 데이터프레임 생성
df = pd.DataFrame({'filename': file_names,'text': all_texts})

# 중복 제거 및 저장
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_excel('pdftext.xlsx')

,filename,text
0,최종보고서(한국수자원공사_2022_2020003110...,"\n보안과제( ), 일반과제(○) / 공개(○), 비공개( ) 발간등록번호..."
1,최종보고서(한국건설생활환경시험연구...,"\n보안 과제( ), 일반 과제( ◯ ) / 공개( ◯ ), 비공개( ..."
2,최종보고서((재)한국화학융합시험연구원...,"- 1 -\n \n보안과제( ), 일반과제(○) / 공개(○), 비공개( )..."
3,최종보고서(서울시립대학교_산학협력단_...,"\n보안과제( ), 일반과제(○) / 공개(○), 비공개( ) 발간등록번호..."
4,최종보고서(강원대학교_산학협력단_2022...,"\n보안과제( ), 일반과제(○) / 공개(○), 비공개( ) 발간등록번호..."
...,...,...
244,f2009020010061_1.pdf,편집순서 2. 제출문\n제 출 문 \n 환경부장관 귀하\n 본 보...
245,"f2007020010042_1(총괄_세부1,2포함).pdf",최종보고서(완결본)\n과제번호 021-051-042\n환경오염 유발물질 대체물질(소...
246,f2007020010054_1.pdf,【별지 제6호서식】\n환경기술개발사업 최종보고서• 초록\n1. 최종보고서 제출서\n...
247,f2002020010010_1.pdf,최종보고서 서식\nⅠ. 인쇄규격\n \n 1. 크기 : 4 × 6 배판 (가로 1...


#### 3. 텍스트 전처리

In [15]:
df['text'] = df['text'].astype(str) # 문자열로 지정
df['text'] = df['text'].str.replace(r'\s+', ' ', regex=True)  # 중복 공백 제거
df['text'] = df['text'].str.replace(r'\·{5,}', ' ', regex=True)  # 연속된 점(·) 제거
df['text'] = df['text'].str.replace(r'\-{5,}', ' ', regex=True) # 연속된 대시(-) 제거
df['text'] = df['text'].str.replace(r'\n', ' ', regex=True) # 줄바꿈을 공백으로 대체
df

,filename,text
0,최종보고서(한국수자원공사_2022_2020003110...,"보안과제( ), 일반과제(○) / 공개(○), 비공개( ) 발간등록번호( ) 미세..."
1,최종보고서(한국건설생활환경시험연구...,"보안 과제( ), 일반 과제( ◯ ) / 공개( ◯ ), 비공개( ) 발간등록번호..."
2,최종보고서((재)한국화학융합시험연구원...,"- 1 - 보안과제( ), 일반과제(○) / 공개(○), 비공개( ) 발간등록번호(..."
3,최종보고서(서울시립대학교_산학협력단_...,"보안과제( ), 일반과제(○) / 공개(○), 비공개( ) 발간등록번호( ) 미세..."
4,최종보고서(강원대학교_산학협력단_2022...,"보안과제( ), 일반과제(○) / 공개(○), 비공개( ) 발간등록번호( ) 미세..."
...,...,...
244,f2009020010061_1.pdf,편집순서 2. 제출문 제 출 문 환경부장관 귀하 본 보고서를 “셀룰로오스를 이용한 ...
245,"f2007020010042_1(총괄_세부1,2포함).pdf",최종보고서(완결본) 과제번호 021-051-042 환경오염 유발물질 대체물질(소재)...
246,f2007020010054_1.pdf,【별지 제6호서식】 환경기술개발사업 최종보고서• 초록 1. 최종보고서 제출서 200...
247,f2002020010010_1.pdf,최종보고서 서식 Ⅰ. 인쇄규격 1. 크기 : 4 × 6 배판 (가로 188mm×세로...


#### 4. 수치형 문자열 추출

##### 4.1 Kiwipiepy

In [ ]:
kiwi = Kiwi() # Kiwi 객체 생성

# 숫자가 포함된 문장 추출 함수
def find_number(text):
    sentences  = kiwi.split_into_sents(text) # kiwi 모델의 문장 분리 기능
    numeric_sentences = [sent.text for sent in sentences if re.search(r'\d+', sent.text)] 
    return numeric_sentences

# 문자열로 인코딩된 리스트를 파이썬 리스트로 변환하는 함수
def convert_to_list(encoded_list):
    try:
        return ast.literal_eval(encoded_list) # 문자열을을 파이썬 리스트로 변환
    except (ValueError, SyntaxError): # 잘못된 형식 예외처리
        print("Invalid format:", encoded_list) 
        return []

# 텍스트에서 특정 키워드가 포함된 수치형 문장 필터링 함수
def get_sentences(text):
    sentences = find_number(text) 

    topic_sentences = []

    if not isinstance(sentences, list): # 리스트 타입인지 확인
        sentences = convert_to_list(sentences) # 리스트가 아니면 변환환

    for sentence in sentences:
        # 지정한 키워드가 문장 내에 포함되어 있다면 리스트에 추가
        if any(keyword.lower() in sentence.lower() for keyword in keyword_list): 
            sentence = re.sub(r'[^\w\s.,%]', ' ', str(sentence))  # 특수 문자, 기호 제거
            sentence = re.sub(r'\s+', ' ', str(sentence)).strip()  # 다중 공백을 단일 공백으로 변경 및 양끝 공백 제거
            topic_sentences.append(sentence) # 필터링 문장 추가
    
    return topic_sentences

##### 4.2 키워드 포함 앞뒤 텍스트 추출

In [ ]:
# 키워드를 포함하는 텍스트를 추출하는 함수
def add_filtered_text_column(text, keyword_list, char_length=80):
    sentences = []
    text = str(text)

    for keyword in keyword_list: 
            start_index = text.find(keyword) # 키워드 시작 위치

            while start_index != -1:  # 키워드가 발견된 경우
                pre_context, post_context = text[:start_index], text[start_index + len(keyword):]
                pre_extract = ''.join(pre_context.split()[-char_length:])  # 공백 제외하고 뒤에서부터 글자 수 세기
                post_extract = ''.join(post_context.split()[:char_length])  # 공백 제외하고 앞에서부터 글자 수 세기
                
                # 키워드 앞뒤 텍스트 포함하여 추출
                extracted_text = f"{pre_extract}{keyword}{post_extract}"
                
                # 수치형 문자열이 포함된 경우만 추가
                if bool(re.search(r'\d+', extracted_text)) is True: 
                  sentences.append(str(extracted_text))

                # 다음 키워드 찾기
                start_index = text.find(keyword, start_index + 1)

    return sentences

In [ ]:
keyword_list=['keyword']
df['filtered_sentence'] = df['text'].apply(get_sentences) # 4.1 방법 적용
df['filtered_text'] = df['text'].apply(lambda text: add_filtered_text_column(str(text), keyword_list)) # 4.2 방법 적용

#### 5. 데이터 저장

In [ ]:
# 허용되지 않는 문자 제거 함수
def remove_illegal_characters(text):
    # 허용되지 않는 문자를 정규 표현식으로 제거
    return re.sub(r'[\x00-\x1F\x7F-\x9F]', '', text)

# 데이터프레임의 모든 텍스트 데이터를 검사하고 수정
df = df.applymap(lambda x: remove_illegal_characters(x) if isinstance(x, str) else x)

df.to_excel('filtered_text.xlsx', index=False, engine='openpyxl')